In [1]:
import codecs
import re

In [2]:
def bigram_to_num(bigram, alphab):
    return alphab.index(bigram[0]) * len(alphab) + alphab.index(bigram[1])

In [3]:
def sort_dict(dicti):
    return dict(sorted(dicti.items(), key=lambda item: item[1]))

In [4]:
def count_bigram_probs(alphab, txt):
    alphab_square = [i + j for i in alphab for j in alphab]

    probs = {}
    length = len(txt) - 1
    for bigram in alphab_square:
        probs[bigram] = txt.count(bigram) / length
    return probs


def count_bigram_probs_2(alphab, txt):
    alphab_square = [i + j for i in alphab for j in alphab]

    probs = {}
    length = int(len(txt) / 2)
    for i in alphab_square:
        probs[i] = 0

    for i in range(0, len(txt), 2):
        if i < len(txt) - 1:
            probs[txt[i] + txt[i + 1]] += 1

    probs = {k: v / length for k, v in probs.items()}
    return probs

In [5]:
def gcdExtended(a, m):
    if a == 0:
        return m, 0, 1
    gcd, x1, y1 = gcdExtended(m % a, a)
    x = y1 - (m // a) * x1
    y = x1
    return gcd, x, y


def inverseModulo(a, m):
    gcd, x, y = gcdExtended(a, m)
    if gcd == 1:
        return (x % m + m) % m
    else:
        return -1


def count_b(x, y, a, m):
    return (y - a * x) % m

In [6]:
def solveEquation(a, b, mod):
    gcd, _, _ = gcdExtended(a, mod)
    if gcd == 1:
        x = ((inverseModulo(a, mod)) * b) % mod
        return x
    elif b % gcd != 0:
        return -1
    else:
        a = a / gcd
        b = b / gcd
        n1 = mod / gcd
        return solveEquation(a, b, n1)

In [7]:
def get_variants(orig, ciphered):
    variants = []
    for i1 in range(len(orig)):
        for j1 in range(len(ciphered)):
            for i2 in range(len(orig)):
                for j2 in range(len(ciphered)):
                    if i1 == i2 or j1 == j2:
                        continue
                    x1y1 = (orig[i1], ciphered[j1])
                    x2y2 = (orig[i2], ciphered[j2])
                    if (x2y2, x1y1) in variants:
                        continue
                    variants.append((x1y1, x2y2))
    return variants

In [8]:
def has_prohibited(text):
    prohibited = [x + 'ь' for x in 'аеуыияьоэюй']
    for pr in prohibited:
        if pr in text:
            return True
    return False

In [9]:
def decipher_bigram(bigram, a, b, alphab):
    m = len(alphab)**2
    n = bigram_to_num(bigram, alphab)
    a_inverse = inverseModulo(a, m)
    return a_inverse*(n - b) % m

def decipher(a, b, txt, alphab):
    alphab_square = [i+j for i in alphab for j in alphab]
    num_to_bigram = {}

    for bigram in alphab_square:
        num_to_bigram[bigram_to_num(bigram, alphab)] = bigram

    deciphered = ''

    bigrams = re.findall('..', txt)
    for bigram in bigrams:
        bigram_d = decipher_bigram(bigram, a, b, alphab)
        deciphered += num_to_bigram[bigram_d]

    return deciphered


In [10]:
def count_roots(most_orig, most_cipher):
    m = 31 * 31
    variants = get_variants(most_orig, most_cipher)
    a_b = []
    for v in variants:
        x1 = bigram_to_num(v[0][0], alphabet)
        x2 = bigram_to_num(v[1][0], alphabet)
        y1 = bigram_to_num(v[0][1], alphabet)
        y2 = bigram_to_num(v[1][1], alphabet)

        x_d = (x1 - x2) % m
        y_d = (y1 - y2) % m

        a = solveEquation(x_d, y_d, m)
        if a != -1:
            b = count_b(x1, y1, a, m)
            if (a, b) not in a_b:
                a_b.append((a, b))
    return a_b

In [11]:
most_orig = ['ст', 'но', 'на', 'то', 'не']

alphabet = list('абвгдежзийклмнопрстуфхцчшщыьэюя')

var_file = codecs.open("./variant.utf8/03.txt", "r", "utf_8_sig")
var_text = var_file.read()
var_file.close()
var_text = re.sub('[^а-я]', '', var_text)
var_text

'кдяхэаюлтдооэтсювнкцябпосбанвооюрретлтцпвоэыохтдшылхщютзгжантзкцхнлюкднхцпвоыомхзотхэтоовцлшвуджозчхйбжьктибэлтцеовбдшйсвцхндншбчбоювнкцябухбюхцхнрбчэшжцюлцлхйостщюшужхриажгцфхзхжцитвожюфпксщхибухкйзюжмьгнхщюзншбхюэотйбавотдцюэшшылхщюабпоябцикбкцывкцхнрбвофишбтдтхыбэляюждзютдлзщюаыпюнозоуюмхэшухэозоихщюкцзоюбзюгсвичхшццнщащцжхщюфмкдвощхщюйуажмздшшшкдысэтмуфьанэйсужушюстлхэдвоэомюфожхетжютдцюгршшкдэйолнойхзозпцэкдютэтнцхыдйщюэтжцтйнбщддцывкцхнцхеоцэвбйбышкдэйюейосежхюбгцэюубйутодткдвощхщющцяюстудвежюнхэджядшищвччощщвунойхзозпцэфтмефпшхтдпощщщыкдвуозеойбдэзэстсдоожмиврбгхнойхзозпцэцэфпэтщощюэоеохсгдюмлзсдвеньрстднтщюфпвцукеоетитмшпнчхшцабшшлсцбухкйэыбдтджюзнхыохнхлхыбэлфошхэдохехвоубпзшбчхлыйбсуодмзеоэотэкшфстднтщюфпкдютэтнцхыдйщюэтвцтйсдлжюасцгцеокочэкдютетэтфтщютздйирэттднттюрюецтйвмшшзцтйищцюеокцфпжюэддйкцвмчоьйнбрбйеинухяуюгкцхнрбвотдмйбарбфшкдэтзэстсдвекдихктщюжонжсиодгуоддйучяожстднтжхщюжощщщыгцщоцпьсждьггжнбгхгцитсдвеоонжзцэюехлцбретйхцпвоыойбщеьжкхшцжосбанолхжжоойераннбйейсвцхнд

In [12]:
bigrams_desc = list(sort_dict(count_bigram_probs(alphabet, var_text)).keys())
bigrams_desc.reverse()

most_var = bigrams_desc[:5]
print(most_var)

roots = count_roots(most_orig, most_var)

for a_b in roots:
    a = a_b[0]
    b = a_b[1]
    deciphered = decipher(a, b, var_text, alphabet)
    if not has_prohibited(deciphered):
        print(f'a = {a}, b = {b}')
        print(deciphered[:100])

['тд', 'рб', 'во', 'щю', 'ет']
a = 199, b = 700
отцеубийствокакизвестноосновноеиизначалыноягрестнглениечеловечестваиотделыноцочеловекавовсякомслучае
